In [ ]:
import numpy as np
import re
import glob
import os
import pickle
from pathlib import Path

def process(file, d, energy=[]):
    with open(file) as i:
        state = "header"
        natoms = 0
        counter = 0
        lines_processed = 0
        files_processed = 0
        o = None

        for line in i:
            lines_processed += 1

            if state == "header":
                natoms = int(line.strip())
                o = open('processed/data.' + str(len(energy)) + '.txt', 'w')
                o.write('# Automatic Generation\n')
                o.write(str(natoms) + ' atoms\n')
                o.write('5 types\n\n')
                state = "energy"
            elif state == "energy":
                energy_val = float(line.strip().split()[2]) / natoms
                energy.append(energy_val)
                o.write('0 ' + str(float(line.strip().split()[3])) + ' xlo xhi\n')
                o.write('0 ' + str(float(line.strip().split()[4])) + ' ylo yhi\n')
                o.write('0 ' + str(float(line.strip().split()[5])) + ' zlo zhi\n')
                o.write('\nAtoms\n\n')
                state = "atoms"
                counter = 0
            elif state == "atoms":
                try:
                    l = line.strip().split()
                    ll = float(l[1])
                except:
                    l = re.findall(r'[A-Za-z]+|-?\d+\.\d+', line.strip())

                o.write(str(counter + 1) + ' ' + str(d[l[0]]) + ' 0 ' + l[1] + ' ' + l[2] + ' ' + l[3] + '\n')
                counter += 1
                if counter == natoms:
                    files_processed += 1
                    o.close()
                    state = "header"

    return energy

def main():
    d = {'C': 6,'H': 1,'Mg': 12,'N': 7,'Zn': 30,'O': 8}
    md = []
    raw_dir = Path('raw')
    processed_dir = Path('processed')
    md = []

    # Create 'processed' directory if it doesn't exist
    processed_dir.mkdir(parents=True, exist_ok=True)

    # Iterate through every file in the 'raw' directory and process it
    files_processed = 0
    for file in raw_dir.iterdir():
        if file.is_file():
            md = process(file, d, energy=md)
    with open('processed_energies_list.pkl', 'wb') as f:
        pickle.dump(md, f)

if __name__ == "__main__":
    main()